In [1]:
import os
import random

import pandas as pd
import haven.db as db 
from mirrorverse.plotting import plot_h3_slider, plot_h3_animation

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [51]:
import numpy as np
import pandas as pd
import plotly.express as px

Ms = np.arange(0.001, 1, 0.01)
ts = np.arange(1, 100, 1)
rows = []
for M in Ms:
    for t in ts:
        rows.append({
            'M': M,
            't': t,
            'C': (1 - (1 - M) ** t) / t,
        })
data = pd.DataFrame(rows).sort_values(['M', 't'], ascending=True)
px.line(data, x='t', y='C', color='M')

In [47]:
T = 500
(1 - (1 - M) ** T) / T

0.002

In [38]:
import numpy as np
import pandas as pd
import plotly.express as px

Ms = np.arange(0.001, 1, 0.05)
rs = np.arange(1, 100, 1)
rows = []
for M in Ms:
    for r in rs:
        rows.append({
            'M': M,
            'r': r,
            'R': (1 - (1 - M) ** (r)),
        })
data = pd.DataFrame(rows).sort_values(['M', 'r'], ascending=True)
px.line(data, x='r', y='R', color='M')

In [32]:
np.log(1-Ms)**2

array([1.00100092e-06, 2.74015401e-03, 1.13363389e-02, 2.67964108e-02,
       5.03528168e-02, 8.35304190e-02, 1.28238859e-01, 1.86902798e-01,
       2.62649773e-01, 3.59588323e-01, 4.83232387e-01, 6.41172362e-01,
       8.44182161e-01, 1.10814225e+00, 1.45760155e+00, 1.93294071e+00,
       2.60645027e+00, 3.62448861e+00, 5.34828263e+00, 9.09586381e+00])

In [17]:
groups = [1.0] * 10
M = [0.5, 1.0] * 5
for day in range(10):
    for i in range(day - 3, day):
        if 0 <= i < len(groups):
            groups[i] = groups[i] * M[i]
print(groups)
sum(groups)

[0.125, 1.0, 0.125, 1.0, 0.125, 1.0, 0.125, 1.0, 0.5, 1.0]


6.0

In [19]:
groups = [1.0] * 10
M = [0.5] * 10
for day in [1, 2, 3, 7, 8, 9]:
    for i in range(day - 3, day):
        if 0 <= i < len(groups):
            groups[i] = groups[i] * M[i]
print(groups)
sum(groups)

[0.125, 0.25, 0.5, 1.0, 0.5, 0.25, 0.125, 0.25, 0.5, 1.0]


4.5

In [2]:
sql = '''
with neighbors as (
    select 
        *,
        row_number() over () as _choice
    from 
        movement_model_neighbors
    where
        max_km = 100
)
select 
    n.current_h3_index,
    n.selected_h3_index,
    n.neighbor_h3_index,
    n.date,
    n._individual,
    n._decision,
    n._choice,
    n.tag_key,
    p.mixed_layer_thickness,
    p.salinity,
    p.temperature,
    p.velocity_east,
    p.velocity_north,
    c.chlorophyll,
    c.nitrate,
    c.phosphate,
    c.silicate,
    c.oxygen,
    c.net_primary_production
from 
    neighbors n 
    inner join copernicus_physics p 
        on p.depth_bin = 25 
        and p.region = 'chinook_study'
        and n.neighbor_h3_index = p.h3_index 
        and n.date = p.date 
    inner join copernicus_biochemistry c
        on c.depth_bin = 25 
        and c.region = 'chinook_study'
        and n.neighbor_h3_index = c.h3_index 
        and n.date = c.date 
'''

cached_file_path = 'cached.snappy.parquet'
if os.path.exists(cached_file_path):
    data = pd.read_parquet(cached_file_path)
else:
    data = db.read_data(sql)
data.to_parquet(cached_file_path)

In [3]:
def set_line_color(row):
    if row['neighbor_h3_index'] == row['current_h3_index']:
        return "orange"
    elif row['neighbor_h3_index'] == row['selected_h3_index']:
        return "purple"
    else:
        return "black"
    
data['color'] = data.apply(set_line_color, axis=1)

In [ ]:
LOCALES = {
    "aleutians": [
        1, 2, 13, 16, 17, 18, 20,
        21, 27, 31, 34, 35, 37,
        41, 60, 62, 64, 68, 72,
        73, 75, 76, 80, 83, 86, 92,
        95, 97, 99, 103, 104,
    ],
    "sitka": [
        3, 7, 9, 11, 24, 26, 30,
        39, 40, 48, 50, 52, 58,
        71, 74, 82, 87, 94, 96,
        100, 106, 109,
    ],
    "cook": [
        4, 14, 15, 43, 51, 55,
        57, 59, 63, 66, 70, 84,
        90, 93, 98, 101, 102,
        105, 108,
    ],
    "ebs": [
        5, 10, 19, 23, 46, 54,
    ],
    "other": [
        6, 29, 44, 
    ],
    "kodiak": [
        8, 12, 22, 28, 32, 38, 42,
        45, 47, 56, 65, 69, 78, 91,

    ],
    "glacier": [
        25, 33, 36, 49, 53, 61,
        67, 77, 79, 81, 85, 88,
        89, 107, 110, 111,
    ]
}

def reverse_dict(d):
    result = {}
    for key, values in d.items():
        for value in values:
            result[value] = key 
    return result

reversed_locales = reverse_dict(LOCALES)
classifications = data[['_individual', 'tag_key']]
classifications['locale'] = classifications['_individual'].apply(lambda i: reversed_locales[i])
data['locale'] = data['_individual'].apply(lambda i: reversed_locales[i])
classifications.to_csv('classifications.csv', index=False)
classifications.head()

In [ ]:
data.columns

## Locales

In [ ]:
data['mod_date'] = data['date'].apply(lambda d: '-'.join(['2020'] + list(d.split('-')[1:])))
data['_time'] = pd.to_datetime(data['mod_date'])
data['_month'] = data['_time'].dt.month
value_col = "_individual"
fig = plot_h3_animation(
    data[(data['color'].isin(['orange']))], 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='_time',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=3, 
    center={"lat": 55, "lon": -165},
    duration=200
)
fig.show()

In [7]:
fig.write_html('common_time.html')

## Mixed Layer Madness

In [ ]:
value_col = 'mixed_layer_thickness'
fig = plot_h3_animation(
    data[data['locale'] == 'glacier'].drop_duplicates(['color', 'neighbor_h3_index', 'date']), 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='date',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=4, 
    center={"lat": 55, "lon": -140},
    duration=200,
    zmax=20
)
fig.show()

In [8]:
fig.write_html('glacier_mixed_layer.html')

## Perhaps Following Net Primary Productivity When Mixing isn't Happening?

In [ ]:
value_col = 'net_primary_production'
data['year'] = pd.to_datetime(data['date']).dt.year
fig = plot_h3_animation(
    data[(data['locale'] == 'aleutians') & (data['year'] > 2019)].drop_duplicates(['color', 'neighbor_h3_index', 'date']), 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='date',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=4, 
    center={"lat": 55, "lon": -165},
    duration=200,
    #zmin=10,
    zmax=20
)
fig.show()

In [ ]:
value_col = 'net_primary_production'
fig = plot_h3_animation(
    data[data['locale'] == 'glacier'].drop_duplicates(['color', 'neighbor_h3_index', 'date']), 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='date',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=4, 
    center={"lat": 55, "lon": -140},
    duration=200,
    zmax=20
)
fig.show()

In [ ]:
value_col = 'net_primary_production'
fig = plot_h3_animation(
    data[data['locale'] == 'sitka'].drop_duplicates(['color', 'neighbor_h3_index', 'date']), 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='date',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=4, 
    center={"lat": 55, "lon": -140},
    duration=200,
    zmax=20
)
fig.show()

## What's Going on in Kodiak/Cook?

In [ ]:
value_col = 'mixed_layer_thickness'
fig = plot_h3_animation(
    data[data['locale'] == 'kodiak'].drop_duplicates(['color', 'neighbor_h3_index', 'date']), 
    value_col=value_col,
    h3_col='neighbor_h3_index',
    slider_col='date',
    line_color_col='color',
    bold_colors=['orange', 'purple'],
    zoom=4, 
    center={"lat": 55, "lon": -140},
    duration=200,
    zmax=20
)
fig.show()